In [11]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    
    for word in words:                
        print "%s\t%s" % (word.lower(), word)

Overwriting mapper.py


In [12]:
%%writefile reducer.py

import sys

current_key = None
name_sum = 0
word_sum = 0

Overwriting reducer.py


In [16]:
%%writefile -a reducer.py

def isName(word):
    return word[0].isupper() and word[1:].islower()
    
def isNameProportion()
    return (100*name_sum/word_sum >= 95)

for line in sys.stdin:
    try:
        key, word = line.strip().split('\t', 1)        
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            if isNameProportion():
                print "%d\t%s" % (name_sum, current_key)
        word_sum = 0
        name_sum = 0
        current_key = key    
    word_sum += 1
    if isName(word):
        name_sum += 1      

if current_key:
    if isNameProportion():
        print "%d\t%s" % (name_sum, current_key)

Appending to reducer.py


In [17]:
%%bash

OUT_DIR="namecount_result"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming name count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

rm: `wordcount_result_1512062266777581': No such file or directory
17/11/30 17:18:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/30 17:18:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/30 17:18:30 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/30 17:18:32 INFO mapreduce.JobSubmitter: number of splits:2
17/11/30 17:18:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1512056254976_0005
17/11/30 17:18:36 INFO impl.YarnClientImpl: Submitted application application_1512056254976_0005
17/11/30 17:18:37 INFO mapreduce.Job: The url to track the job: http://957a16de490c:8088/proxy/application_1512056254976_0005/
17/11/30 17:18:37 INFO mapreduce.Job: Running job: job_1512056254976_0005
17/11/30 17:19:34 INFO mapreduce.Job: Job job_1512056254976_0005 running in uber mode : false
17/11/30 17:19:34 INFO mapreduce.Job:  map 0% reduce 0%
17/11/30 17:20:44 INFO mapreduce.Job:  map 2% reduce 0%
17/11/30 17:20